## Data Cleaning and Validation
In this challenge, I will clean the data for a dataset of article open-access prices paid (Sourced: WELLCOME Trust between 2012-2013). 


Once cleaned, I will determine:


1) The five most common journals and total articles for each

2) The mean, median, and standard deviation of the open-access cost per article per journal

In [1]:
import pandas as pd
import numpy as np

### Importing, encoding
Below, I used encoding iso-8859-1 as opposed to the normal UTF-8 because there appears to be some latin characters that the latter was unable to register.

In [2]:
df = pd.read_csv('journal-data/WELLCOME_APCspend2013_forThinkful.csv', encoding='iso-8859-1')

In [3]:
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [4]:
df.columns

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'],
      dtype='object')

### Renaming columns
Adding underscores to spaces, and simplifying the cost column name

In [5]:
df.columns = ['PMID/PMCID','Publisher','Journal_title','Article_title','Cost']

In [6]:
df.describe()

,PMID/PMCID,Publisher,Journal_title,Article_title,Cost
count,1928,2127,2126,2127,2127
unique,1880,299,984,2126,1402
top,-,Elsevier,PLoS One,"Exclusive breastfeeding, diarrhoel morbidity a...",£2040.00
freq,7,387,92,2,94


### Remove currency signs from prices

In [37]:
df.Cost = df.Cost.str.replace('£', '')
df.Cost = df.Cost.str.replace('$', '')

### Analyzing journal IDs

In [8]:
df['PMID/PMCID'].value_counts(ascending=True)

23458425 PMC3630740                                 1
PMC3777123                                          1
23201205                                            1
3676539                                             1
PMC3460760                                          1
PMCID: PMC3491471                                   1
PMC3546795                                          1
PMCID:\n    PMC3761558                              1
PMID: 23681165  PMC3824307                          1
PMC3169830                                          1
PMC3557262                                          1
PMC3472025                                          1
PMC3507630                                          1
PMC3374517                                          1
PMC3719211\n\n                                      1
PMID: 23140835 PMC3625108                           1
PMC3747673\n\n                                      1
PMC3621106                                          1
PMID: 23477577 PMC3727344   

In [9]:
df.loc[df['PMID/PMCID']=='41609']

,PMID/PMCID,Publisher,Journal_title,Article_title,Cost
950,41609,John Wiley & Sons Ltd,Journal of Medical Virology,High level of HIV-2 false positivity in KwaZul...,612.59
1956,41609,Wiley,Journal of Cellular Biochemistry,The U5 snRNA internal loop 1 is a platform for...,1971.87


Despite the same PMID, they are not the same articles. For our purposes here, we won't gain any valuable information from the PMID/PMCID column so I will drop it.

In [10]:
df = df.drop(['PMID/PMCID'], axis=1)

### Remove duplicate entries

In [11]:
# remove any entries with indentical values
df = df.drop_duplicates()

In [12]:
df.describe()

,Publisher,Journal_title,Article_title,Cost
count,2126,2125,2126,2126
unique,299,984,2126,1402
top,Elsevier,PLoS One,CD6 attenuates early and late signaling events...,2040.00
freq,387,91,1,94


### Merge journal title entries together that have different spelling/capitalizations

There are 984 unique journal title's in this dataset. How can I reasonably sort through it all?

In [13]:
df['Journal_title'].value_counts()

PLoS One                                            91
PLoS ONE                                            62
Journal of Biological Chemistry                     48
Nucleic Acids Research                              21
Proceedings of the National Academy of Sciences     19
PLoS Neglected Tropical Diseases                    18
Human Molecular Genetics                            18
Nature Communications                               17
Neuroimage                                          15
PLoS Pathogens                                      15
PLoS Genetics                                       15
Brain                                               14
NeuroImage                                          14
PLOS ONE                                            14
BMC Public Health                                   14
Movement Disorders                                  13
Developmental Cell                                  12
Biochemical Journal                                 12
Journal of

In [21]:
# integrate articles that are spelled differently (acronym or otherwise, caps or not)
df.Journal_title = df.Journal_title.replace({'PLoS ONE':'PLoS One',
                                             'PLOS ONE':'PLoS One',
                                             'Plos ONE':'PLos One',
                                             'PloS One':'PLoS One',
                                             'PLOS One':'PLoS One'})

In [22]:
df['Journal_title'].value_counts()

PLoS One                                                   178
Journal of Biological Chemistry                             48
Nucleic Acids Research                                      21
Proceedings of the National Academy of Sciences             19
PLoS Neglected Tropical Diseases                            18
Human Molecular Genetics                                    18
Nature Communications                                       17
PLoS Genetics                                               15
Neuroimage                                                  15
PLoS Pathogens                                              15
NeuroImage                                                  14
BMC Public Health                                           14
Brain                                                       14
Movement Disorders                                          13
Journal of Neuroscience                                     12
Developmental Cell                                     

In [26]:
df.Journal_title = df.Journal_title.str.strip()

In [27]:
df.Journal_title.unique

<bound method Series.unique of 0                                  Psychological Medicine
1                                       Biomacromolecules
2                                              J Med Chem
3                                              J Med Chem
4                                              J Org Chem
5                          Journal of Medicinal Chemistry
6                            Journal of Proteome Research
7                                               Mol Pharm
8                                    ACS Chemical Biology
9                                    ACS Chemical Biology
10           Journal of Chemical Information and Modeling
11                         Journal of Medicinal Chemistry
12                         Journal of Medicinal Chemistry
13                                           Biochemistry
14                         Journal of Medicinal Chemistry
15                                       Gastroenterology
16                        Journal of Biol

I'm not sure why there are still multiple duplicates in journal titles. Many of them look exactly the same, and I don't anticipate white spaces being an issue as I've removed them from either side. In any case, I've combined same journal entries in which a name variant has greater than 8 entries. Therefore, any duplicates that remain are only found in small amounts - it would not change the most popularly published journal in this dataset.

### Change cost type from object to float

In [35]:
df.Cost.dtype

dtype('O')

In [38]:
df.Cost = pd.to_numeric(df.Cost)

In [39]:
df.describe()

,Cost
count,2126.000000
mean,24078.272079
std,146894.351294
min,0.000000
25%,1280.000000
50%,1884.155000
75%,2321.342500
max,999999.000000


In [44]:
df = df[df.Cost < 10000]

In [45]:
df.loc[df.Cost>2321]

,Publisher,Journal_title,Article_title,Cost
1,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04
5,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,2392.20
6,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,2367.95
18,American Association of Immunologists,Journal of Immunology,Regulation of Foxp3+ Inducible Regulatory T Ce...,3108.08
35,American College of Chest Physicians,CHEST,Synthetic response of stimulated respiratory e...,2383.94
43,American Psychiatric Association,American Journal of Psychiatry,Methamphetamine-induced disruption of frontost...,2351.73
44,American Psychiatric Publishing,Americal Journal of Psychiatry,High Loading of Polygenic Risk for ADHD in Chi...,2350.02
45,American Psychological Association,Behavioral Neuroscience,Evidence that the rat hippocampus has contrast...,3021.72
46,American Psychological Association,Behavioral Neuroscience,Dissociation of recognition and recency memory...,3209.63
47,American Psychological Association,Behavioral Neuroscience,The Neural Basis of Nonvisual Object Recognition,3211.77


In [49]:
df.describe()

,Cost
count,2076.000000
mean,1822.535857
std,767.571773
min,0.000000
25%,1268.700000
50%,1851.650000
75%,2302.330000
max,6000.000000


In [50]:
df.median()

Cost    1851.65
dtype: float64

I don't know this title below so I can't be sure that the cost is incorrect. Will assume it's just a particularly expensive journal.

In [48]:
df.loc[df.Cost==6000]

,Publisher,Journal_title,Article_title,Cost
1619,public.service.co.uk,Public Service Review,Laboratory Science in Tropical Medicine,6000.0


### Concluding answers
1) The five most common journals and total articles for each
- PLoS One,                                                   178
- Journal of Biological Chemistry,                             48
- Nucleic Acids Research,                                      21
- Proceedings of the National Academy of Sciences,             19
- PLoS Neglected Tropical Diseases,                            18


2) The mean, median, and standard deviation of the open-access cost per article per journal
- Mean: £1,822.54
- Median: £1,851.65
- Standard deviation: £767.57

Bonus: identify open-access prices by subject area
- I'll go over the logic of completing this. I would make an entirely new dataframe with just the Publisher, Journal title, and Cost. Then, I would use RegEx to find all entries under Publisher with a given subject name (ex. 'chemistry', 'biology', 'astronomy' etc.) and add that subject name to a new column, 'Subject.' For those that publisher could not label, I would then try Journal title. Once the subjects are labeled, I would take the average of the cost of any given subject label.